### setup

In [1]:
from cvxpy import *
from datetime import datetime
from EbolaSimulator import*
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
import time

%matplotlib inline

%load_ext autoreload
%autoreload 2

### load graph model from file

In [2]:
graph = pickle.load(open(os.getcwd()+'/ebola_graph.pkl' ,'rb'))

### parameters 

In [51]:
alpha = 0.14
beta = (1/1.17)*0.15
parameters = {}
parameters['alpha'] = alpha
parameters['beta'] = beta

gamma = 0.9
sigma = -1
capacity = 5
trigger_methods = 5

### no control simulation

In [63]:
folder = '/no_control/'

sim = EbolaSimulator(graph, init, parameters, rng=1)

plt.figure(figsize=(10,10))
plt.axis('off')
for k in sim.states.keys():
    country, location = k
    
    if sim.states[k]['status'] == 'susceptible':
        color = 'g'
    elif sim.states[k]['status'] == 'infected':
        color = 'r'
    elif sim.states[k]['status'] == 'removed':
        color = 'k'
    
    #if country == 'sierra leone':
    #    color = 'r'
    #elif country == 'guinea':
    #    color = 'g'
    #elif country == 'liberia':
    #    color = 'b'
        
    pos = graph[k]['pos']
    markersize = 10
    plt.plot(*pos, marker='o', 
             markeredgecolor=color, markerfacecolor='w', markeredgewidth=2, markersize=markersize)
    
    for el in graph[k]['edges']:
        other_pos = graph[el]['pos']
        plt.plot([pos[0], other_pos[0]], [pos[1], other_pos[1]], zorder=0, color='gray', linewidth=1)
        
    plt.text(-1300, 600, 'WEEK %3d' %sim.iter, fontsize=20)
    
fn = os.getcwd() + folder + 'img%s' %str(sim.iter).zfill(3)
plt.savefig(fn, dpi=300, bbox_inches='tight')
plt.close()

init = {}
init[('guinea','gueckedou')] = 1

while not sim.end:
    action = []
    
    #sim.step(action)
    
    plt.figure(figsize=(10,10))
    plt.axis('off')
    for k in sim.states.keys():
        country, location = k
        
        if sim.states[k]['status'] == 'susceptible':
            color = 'g'
        elif sim.states[k]['status'] == 'infected':
            color = 'r'
        elif sim.states[k]['status'] == 'removed':
            color = 'k'

        #if country == 'sierra leone':
        #    color = 'r'
        #elif country == 'guinea':
        #    color = 'g'
        #elif country == 'liberia':
        #    color = 'b'

        pos = graph[k]['pos']
        markersize = sim.states[k]['value']/10.0 + 10
        plt.plot(*pos, marker='o', 
             markeredgecolor=color, markerfacecolor='w', markeredgewidth=2, markersize=markersize)

        for el in graph[k]['edges']:
            other_pos = graph[el]['pos']
            plt.plot([pos[0], other_pos[0]], [pos[1], other_pos[1]], zorder=0, color='gray', linewidth=1)
            
    for a in action:
        pos = graph[a]['pos']
        plt.plot(*pos, marker='o', color = 'b', markersize=5)
        
    plt.text(-1300, 600, 'WEEK %3d' %sim.iter, fontsize=20)

    fn = os.getcwd() + folder + 'img%s' %str(sim.iter).zfill(3)
    plt.savefig(fn, dpi=300, bbox_inches='tight')
    plt.close()
    
    sim.step(action)
    
for t_extra in range(10):
    plt.figure(figsize=(10,10))
    plt.axis('off')
    for k in sim.states.keys():
        country, location = k
    
        if sim.states[k]['status'] == 'susceptible':
            color = 'g'
        elif sim.states[k]['status'] == 'infected':
            color = 'r'
        elif sim.states[k]['status'] == 'removed':
            color = 'k'
            
        #if country == 'sierra leone':
        #    color = 'r'
        #elif country == 'guinea':
        #    color = 'g'
        #elif country == 'liberia':
        #    color = 'b'

        pos = graph[k]['pos']
        markersize = sim.states[k]['value']/10.0 + 10
        plt.plot(*pos, marker='o', 
             markeredgecolor=color, markerfacecolor='w', markeredgewidth=2, markersize=markersize)

        for el in graph[k]['edges']:
            other_pos = graph[el]['pos']
            plt.plot([pos[0], other_pos[0]], [pos[1], other_pos[1]], zorder=0, color='gray', linewidth=1)
            
    for a in action:
        pos = graph[a]['pos']
        plt.plot(*pos, marker='o', color = 'b', markersize=5)
        
    plt.text(-1300, 600, 'WEEK %3d' %sim.iter, fontsize=20)

    fn = os.getcwd() + folder + 'img%s' %str(sim.iter+t_extra).zfill(3)
    plt.savefig(fn, dpi=300, bbox_inches='tight')
    plt.close()  

### approximate Qw simulation

In [37]:
weights_Qw = [0.9824708, 0.67315834, -2.63809994, 0.07975353]

def Qw(states, graph, capacity, weights, alpha, beta, gamma, sigma):
    control = []
    for k in states.keys():
        if states[k]['status'] == 'infected':
            ni = 0
            for el in graph[k]['edges']:
                if states[el]['status'] == 'susceptible':
                    ni += 1
                    #nj = 0
                    #for ek in graph[el]['edges']:
                    #    if states[ek]['status'] == 'infected':
                    #        nj += 1
                    #ni += (1-alpha*nj)
            
            #value = gamma*weights[3]*ni
            value = weights[3]*ni
            control.append((value,k))
            
    control = sorted(control, key=lambda tup: tup[0], reverse=True)[:capacity]
    control = [el[1] for el in control]
            
    return control

In [62]:
folder = '/approximate_qw/'

sim = EbolaSimulator(graph, init, parameters, rng=1)

plt.figure(figsize=(10,10))
plt.axis('off')
for k in sim.states.keys():
    country, location = k
    
    if sim.states[k]['status'] == 'susceptible':
        color = 'g'
    elif sim.states[k]['status'] == 'infected':
        color = 'r'
    elif sim.states[k]['status'] == 'removed':
        color = 'k'
    
    #if country == 'sierra leone':
    #    color = 'r'
    #elif country == 'guinea':
    #    color = 'g'
    #elif country == 'liberia':
    #    color = 'b'
        
    pos = graph[k]['pos']
    markersize = 10
    plt.plot(*pos, marker='o', 
             markeredgecolor=color, markerfacecolor='w', markeredgewidth=2, markersize=markersize)
    
    for el in graph[k]['edges']:
        other_pos = graph[el]['pos']
        plt.plot([pos[0], other_pos[0]], [pos[1], other_pos[1]], zorder=0, color='gray', linewidth=1)
        
    plt.text(-1300, 600, 'WEEK %3d' %sim.iter, fontsize=20)
    
fn = os.getcwd() + folder + 'img%s' %str(sim.iter).zfill(3)
plt.savefig(fn, dpi=300, bbox_inches='tight')
plt.close()

init = {}
init[('guinea','gueckedou')] = 1

while not sim.end:
    action = []
    if sim.iter > trigger_methods:
        action = Qw(sim.states, graph, capacity, weights_Qw, alpha, beta, gamma, sigma)
    
    #sim.step(action)
    
    plt.figure(figsize=(10,10))
    plt.axis('off')
    for k in sim.states.keys():
        country, location = k
        
        if sim.states[k]['status'] == 'susceptible':
            color = 'g'
        elif sim.states[k]['status'] == 'infected':
            color = 'r'
        elif sim.states[k]['status'] == 'removed':
            color = 'k'

        #if country == 'sierra leone':
        #    color = 'r'
        #elif country == 'guinea':
        #    color = 'g'
        #elif country == 'liberia':
        #    color = 'b'

        pos = graph[k]['pos']
        markersize = sim.states[k]['value']/10.0 + 10
        plt.plot(*pos, marker='o', 
             markeredgecolor=color, markerfacecolor='w', markeredgewidth=2, markersize=markersize)

        for el in graph[k]['edges']:
            other_pos = graph[el]['pos']
            plt.plot([pos[0], other_pos[0]], [pos[1], other_pos[1]], zorder=0, color='gray', linewidth=1)
            
    for a in action:
        pos = graph[a]['pos']
        plt.plot(*pos, marker='o', color = 'b', markersize=5)
        
    plt.text(-1300, 600, 'WEEK %3d' %sim.iter, fontsize=20)

    fn = os.getcwd() + folder + 'img%s' %str(sim.iter).zfill(3)
    plt.savefig(fn, dpi=300, bbox_inches='tight')
    plt.close()
    
    sim.step(action)
    
for t_extra in range(10):
    plt.figure(figsize=(10,10))
    plt.axis('off')
    for k in sim.states.keys():
        country, location = k
    
        if sim.states[k]['status'] == 'susceptible':
            color = 'g'
        elif sim.states[k]['status'] == 'infected':
            color = 'r'
        elif sim.states[k]['status'] == 'removed':
            color = 'k'
            
        #if country == 'sierra leone':
        #    color = 'r'
        #elif country == 'guinea':
        #    color = 'g'
        #elif country == 'liberia':
        #    color = 'b'

        pos = graph[k]['pos']
        markersize = sim.states[k]['value']/10.0 + 10
        plt.plot(*pos, marker='o', 
             markeredgecolor=color, markerfacecolor='w', markeredgewidth=2, markersize=markersize)

        for el in graph[k]['edges']:
            other_pos = graph[el]['pos']
            plt.plot([pos[0], other_pos[0]], [pos[1], other_pos[1]], zorder=0, color='gray', linewidth=1)
            
    for a in action:
        pos = graph[a]['pos']
        plt.plot(*pos, marker='o', color = 'b', markersize=5)
        
    plt.text(-1300, 600, 'WEEK %3d' %sim.iter, fontsize=20)

    fn = os.getcwd() + folder + 'img%s' %str(sim.iter+t_extra).zfill(3)
    plt.savefig(fn, dpi=300, bbox_inches='tight')
    plt.close()  
